In [1]:
from datetime import datetime
import os
import json
import requests
import hmac
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
from scipy.stats import linregress
import time
from config import API_KEY
from client import FtxClient

ModuleNotFoundError: No module named 'ciso8601'

In [11]:
url = 'https://ftx.com/api/markets'

#get BTCUSD spot price
base_currency = 'BTC'
quote_currency = 'USD'

#construct request url
request_url = f"{url}/{base_currency}/{quote_currency}"

btcusd_df = pd.DataFrame(requests.get(request_url).json())
btcusd_df['result']

#define resolutions
hourly = str(60*60)
daily = str(60*60*24)


In [78]:
#define function to fetch multiple months of data for 2021
def historical2021_hourly(m, d, n_months):
    """ grabs multiple months of hourly data for 2021 by sending multiple queries to FTX API
    
    Parameters:
        m(int): month of year
        d(int): day of week
        n_months(int): number of months from starting date 
                        (ex. input n_months = 3 if you want 3 months of data beginning from start date)
        
    Returns:
        df_2022_hr(pd.DataFrame): pandas dataframe containing all data for specified number of months since start date 
    
    """
    while n_months > 0:
        #fetch data for the first month beginning with the start date 
        if m == m:
            
            start_date = datetime(2021, m, d).timestamp()
            end_date = datetime(2021, m+1, d).timestamp()
            
            historical = requests.get(
                f'{request_url}/candles?resolution={hourly}&start_time={start_date}&end_time={end_date}'
                ).json()
            
            df_2021_hr = pd.DataFrame(historical['result'])
                        
            #add 1 to starting month
            m += m
            
        
        #fetch data for the next month
        else:
            start_date = datetime(2021, m, d).timestamp()
            end_date = datetime(2021, m+1, d).timestamp()
            
            historical = requests.get(
                f'{request_url}/candles?resolution={hourly}&start_time={start_date}&end_time={end_date}'
                ).json()
            
            #convert json to pd df_2022_hr and append all rows except for the first repeating row to df_2022_hr
            next_month = pd.DataFrame(historical['result'])
            df_2021_hr.append(next_month.iloc[1: , :])
            
            m += m
            
        #subtract n_months counter
        n_months -= n_months
    else:
        return df_2021_hr

#runs function and saves return df_2022_hr to df_2021
df_2021_hr = historical2021_hourly(1,1,3)  

#convert to datetime and drop unnecessary rows df_2021
df_2021_hr['date'] = pd.to_datetime(df_2021_hr['time']/1000, unit='s', origin='unix')

df_2021_hr.drop(['startTime', 'time'], axis=1, inplace=True)

df_2021_hr


,open,high,low,close,volume,date
0,29280.5,29296.0,29146.5,29240.0,5.523795e+06,2021-01-01 05:00:00
1,29240.0,29315.0,29127.5,29231.0,7.436047e+06,2021-01-01 06:00:00
2,29231.0,29241.5,28887.0,29168.0,8.430755e+06,2021-01-01 07:00:00
3,29168.0,29251.0,28928.5,29061.0,7.003067e+06,2021-01-01 08:00:00
4,29061.0,29379.5,29041.5,29275.0,5.764370e+06,2021-01-01 09:00:00
...,...,...,...,...,...,...
740,32575.0,33563.0,32511.0,33448.0,9.540823e+06,2021-02-01 01:00:00
741,33448.0,33858.0,33243.0,33672.0,6.190308e+06,2021-02-01 02:00:00
742,33672.0,33779.0,33488.0,33592.0,3.189237e+06,2021-02-01 03:00:00
743,33592.0,33953.0,33454.0,33578.0,4.414067e+06,2021-02-01 04:00:00


In [80]:
def historical2021_daily(m, d, n_months):
    while n_months > 0:
        #fetch data for the first month beginning with the start date 
        if m == m:
            
            start_date = datetime(2021, m, d).timestamp()
            end_date = datetime(2021, m+1, d).timestamp()
            
            historical = requests.get(
                f'{request_url}/candles?resolution={daily}&start_time={start_date}&end_time={end_date}'
                ).json()
            
            df_2021_dy = pd.DataFrame(historical['result'])
                        
            #add 1 to starting month
            m += m
        
        #fetch data for the next month
        else:
            start_date = datetime(2021, m, d).timestamp()
            end_date = datetime(2021, m+1, d).timestamp()
            
            historical = requests.get(
                f'{request_url}/candles?resolution={daily}&start_time={start_date}&end_time={end_date}'
                ).json()
            
            #convert json to pd df_2022_hr and append all rows except for the first repeating row to df_2022_hr
            next_month = pd.DataFrame(historical['result'])
            df_2021_dy.append(next_month.iloc[1: , :])
            
            
            #add 1 to starting month
            m += m
        #subtract n_months counter
        n_months -= n_months
    else:
        return df_2021_dy

          
df_2021_dy = historical2021_daily(1,1,4)  

#convert to datetime and drop unnecessary rows df_2021
df_2021_dy['date'] = pd.to_datetime(df_2021_dy['time']/1000, unit='s', origin='unix')

df_2021_dy.drop(['startTime', 'time'], axis=1, inplace=True)

df_2021_dy        
            

,open,high,low,close,volume,date
0,29399.0,33349.5,29023.5,32201.5,2.544018e+08,2021-01-02
1,32201.5,34809.5,31978.0,33045.5,1.470195e+08,2021-01-03
2,33045.5,33658.5,27700.5,32005.0,1.782005e+08,2021-01-04
3,32005.0,34499.5,29923.5,34034.5,1.324108e+08,2021-01-05
4,34034.5,37038.0,33367.0,36871.5,1.093933e+08,2021-01-06
5,36871.5,40428.0,36260.0,39485.0,1.923478e+08,2021-01-07
6,39485.0,42000.0,36548.0,40625.0,1.890426e+08,2021-01-08
7,40625.0,41410.0,38780.0,40234.0,7.512709e+07,2021-01-09
8,40234.0,41447.0,35001.0,38162.0,1.911767e+08,2021-01-10
9,38158.0,38330.0,30255.0,35484.0,4.192866e+08,2021-01-11


In [64]:
#define function to fetch multiple months of data for 2021
def historical2022_hourly(m, d, n_months):
    """ grabs multiple months of hourly data for 2021 by sending multiple queries to FTX API
    
    Parameters:
        m(int): month of year
        d(int): day of week
        n_months(int): number of months from starting date 
                        (ex. input n_months = 3 if you want 3 months of data beginning from start date)
        
    Returns:
        df_2022_hr(pd.DataFrame): pandas dataframe containing all data for specified number of months since start date 
    
    """
    while n_months > 0:
        #fetch data for the first month beginning with the start date 
        if m == m:
            
            start_date = datetime(2022, m, d).timestamp()
            end_date = datetime(2022, m+1, d).timestamp()
            
            historical = requests.get(
                f'{request_url}/candles?resolution={hourly}&start_time={start_date}'
                ).json()
            
            df_2022_hr = pd.DataFrame(historical['result'])
                        
            #add 1 to starting month
            m += m
            
        
        #fetch data for the next month
        else:
            start_date = datetime(y, m, d).timestamp()
            end_date = datetime(y, m+1, d).timestamp()
            
            historical = requests.get(
                f'{request_url}/candles?resolution={hourly}&start_time={start_date}&end_time={end_date}'
                ).json()
            
            #convert json to pd df_2022_hr and append all rows except for the first repeating row to df_2022_hr
            next_month = pd.DataFrame(historical['result'])
            df_2022_hr.append(next_month.iloc[1: , :])
            
            m += m
            
        #subtract n_months counter
        n_months -= n_months
    else:
        return df_2022_hr

#runs function and saves return df_2022_hr to df_2021
df_2022_hr = historical2022_hourly(1,1,3)  

#convert to datetime and drop unnecessary rows df_2021
df_2022_hr['date'] = pd.to_datetime(df_2022_hr['time']/1000, unit='s', origin='unix')

df_2022_hr.drop(['startTime', 'time'], axis=1, inplace=True)

df_2022_hr

,open,high,low,close,volume,date
0,29971.0,30076.0,29932.0,30034.0,7.097016e+06,2022-06-05 19:00:00
1,30034.0,30110.0,29917.0,29923.0,1.157194e+07,2022-06-05 20:00:00
2,29923.0,29960.0,29792.0,29937.0,9.600126e+06,2022-06-05 21:00:00
3,29937.0,30055.0,29847.0,30000.0,1.933518e+07,2022-06-05 22:00:00
4,30000.0,30025.0,29867.0,29897.0,1.128383e+07,2022-06-05 23:00:00
...,...,...,...,...,...,...
1495,22915.0,23009.0,22911.0,23000.0,4.488902e+06,2022-08-07 02:00:00
1496,23000.0,23045.0,22977.0,22993.0,6.132984e+06,2022-08-07 03:00:00
1497,22993.0,23029.0,22944.0,22951.0,5.817889e+06,2022-08-07 04:00:00
1498,22951.0,22999.0,22950.0,22971.0,3.709893e+06,2022-08-07 05:00:00


In [77]:
df_2022_hr['ohlc4'] = (df_2022_hr['open']+df_2022_hr['high']+df_2022_hr['low']+df_2022_hr['close'])/4

df_2022_hr['SMA'] = df_2022_hr['ohlc4'].rolling(window = 48).mean()
df_2022_hr = df_2022_hr[['date','open','high','low','close','ohlc4','volume','SMA']]

df_2022_hr

,date,open,high,low,close,ohlc4,volume,SMA
0,2022-06-05 19:00:00,29971.0,30076.0,29932.0,30034.0,30003.25,7.097016e+06,NaN
1,2022-06-05 20:00:00,30034.0,30110.0,29917.0,29923.0,29996.00,1.157194e+07,NaN
2,2022-06-05 21:00:00,29923.0,29960.0,29792.0,29937.0,29903.00,9.600126e+06,NaN
3,2022-06-05 22:00:00,29937.0,30055.0,29847.0,30000.0,29959.75,1.933518e+07,NaN
4,2022-06-05 23:00:00,30000.0,30025.0,29867.0,29897.0,29947.25,1.128383e+07,NaN
...,...,...,...,...,...,...,...,...
1495,2022-08-07 02:00:00,22915.0,23009.0,22911.0,23000.0,22958.75,4.488902e+06,23139.031250
1496,2022-08-07 03:00:00,23000.0,23045.0,22977.0,22993.0,23003.75,6.132984e+06,23136.677083
1497,2022-08-07 04:00:00,22993.0,23029.0,22944.0,22951.0,22979.25,5.817889e+06,23132.708333
1498,2022-08-07 05:00:00,22951.0,22999.0,22950.0,22971.0,22967.75,3.709893e+06,23128.640625


In [76]:
# create a graph object
fig = go.Figure()

#set up layout
fig.update_layout(
    title={
        'text' : "BTCUSD",
        "x" : 0.5,
        'xanchor': 'center'
    },
    xaxis_title="Date",
    yaxis_title='Price',
    xaxis_rangeslider_visible=False
)

#add candletstick
fig.add_trace(
    go.Candlestick(
        x=df_2022_hr['date'],
        open=df_2022_hr['open'],
        high=df_2022_hr['high'],
        low=df_2022_hr['low'],
        close=df_2022_hr['close']
    )
)

fig.add_trace(
    go.scatter(
        x=df_2021_hr['date'],
        y=df_2021_hr['SMA']
        
    )
    
    
)

fig.show()

KeyError: 'SMA'